In [13]:
import os
import seaborn as sns
import h5py
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import time

save_outputs = True

In [14]:
sns.set_context('talk')
sns.set_style('white')

In [18]:
dataset = 'free-mouse'

if dataset == 'dipoppa':
#     hdf5_file = '/media/mattw/data/conditional-aes/dipoppa/pupil/MD0ST5/session-4/data.hdf5'
#     save_file = '/media/mattw/data/conditional-aes/dipoppa/pupil/MD0ST5/session-4/check.mp4'
    hdf5_file = '/media/mattw/data/conditional-aes/dipoppa/pupil/SB028/session-6/data.hdf5'
    save_file = '/media/mattw/data/conditional-aes/dipoppa/pupil/SB028/session-6/check.mp4'
    offset_trial = 0
    max_frames = 1000
elif dataset == 'musall':
#     hdf5_file = '/media/mattw/data/conditional-aes/musall/vistrained/mSM30/10-Oct-2017/data.hdf5'
#     save_file = '/media/mattw/data/conditional-aes/musall/vistrained/mSM30/10-Oct-2017/check.mp4'
#     hdf5_file = '/media/mattw/data/musall/vistrained/mSM36/05-Dec-2017/data.hdf5'
#     save_file = '/media/mattw/data/musall/vistrained/mSM36/05-Dec-2017/check.mp4'
#     hdf5_file = '/media/mattw/data/conditional-aes/musall/vistrained/mSM36/05-Dec-2017-wpaw/data.hdf5'
#     save_file = '/media/mattw/data/conditional-aes/musall/vistrained/mSM36/05-Dec-2017-wpaw/check.mp4'
    hdf5_file = '/media/mattw/data/conditional-aes/musall/vistrained/mSM36/05-Dec-2017/data.hdf5'
    save_file = '/media/mattw/data/conditional-aes/musall/vistrained/mSM36/05-Dec-2017/check.mp4'
    offset_trial = 0
    max_frames = 600
elif dataset == 'ibl2':
    hdf5_file = '/media/mattw/data/conditional-aes/ibl/ephys/animal-0/192_coarse-66/data.hdf5'
    save_file = '/media/mattw/data/conditional-aes/ibl2/ephys/animal-0/192_coarse-66/check.mp4'
    offset_trial = 0
    max_frames = 600
elif dataset == 'open-field':
    hdf5_file = '/media/mattw/data/conditional-aes/rodriguez/open-field/B96/sess-01/data.hdf5'
    save_file = '/media/mattw/data/conditional-aes/rodriguez/open-field/B96/sess-01/check.mp4'
    offset_trial = 0
    max_frames = 400
elif dataset == 'free-mouse':
    hdf5_file = '/Users/johnzhou/code/behavenet/results/rodriguez.hdf5'
    save_file = '/Users/johnzhou/code/behavenet/results/check.mp4'
    offset_trial = 0
    max_frames = 400
else:
    raise Exception

ims_orig = []
latents = []
labels_sc = []
masks = []

# load data
with h5py.File(hdf5_file, 'r', libver='latest', swmr=True) as file:
    n_trials = len(file['images'])
    n_frames = 0
    for trial in range(n_trials):
        trial_ = trial + offset_trial
        ims_orig.append(file['images'][str('trial_%04i' % trial_)][()].astype('float32'))
        latents.append(file['labels'][str('trial_%04i' % trial_)][()].astype('float32'))
        if 'labels_sc' in list(file.keys()):
            labels_sc.append(file['labels_sc'][str('trial_%04i' % trial_)][()].astype('float32'))
        if 'labels_masks' in list(file.keys()):
            masks.append(file['labels_masks'][str('trial_%04i' % trial_)][()].astype('float32'))
        n_frames += latents[-1].shape[0]
        if n_frames >= max_frames:
            break
    
ims_orig = np.vstack(ims_orig) / 255
latents = np.vstack(latents)
labels_sc = np.vstack(labels_sc)
masks = np.vstack(masks)
print('n_trials: %i' % n_trials)

n_trials: 185


In [16]:
print(labels_sc.shape)
print(np.amax(labels_sc, axis=0) - np.amin(labels_sc, axis=0))
for i in range(labels_sc.shape[-1]):
    print('INDEX: {}'.format(i))
    print(np.histogram(labels_sc[:, i], bins=10))

(23560, 8)
[160. 192. 159.  94. 216. 155. 138. 179.]
INDEX: 0
(array([  133,     0,     0,     0,     0,     0,   159,  2232, 14308,
        6728]), array([  0.,  16.,  32.,  48.,  64.,  80.,  96., 112., 128., 144., 160.],
      dtype=float32))
INDEX: 1
(array([  267,     0,     4,     7,    74,  2422, 10542,  9673,   553,
          18]), array([  0. ,  19.2,  38.4,  57.6,  76.8,  96. , 115.2, 134.4, 153.6,
       172.8, 192. ], dtype=float32))
INDEX: 2
(array([  341,     0,     0,     0,    37, 12479, 10593,    49,    29,
          32]), array([  0. ,  15.9,  31.8,  47.7,  63.6,  79.5,  95.4, 111.3, 127.2,
       143.1, 159. ], dtype=float32))
INDEX: 3
(array([  550,     0,     0,   215,  4300, 11342,  5203,  1604,   313,
          33]), array([ 0. ,  9.4, 18.8, 28.2, 37.6, 47. , 56.4, 65.8, 75.2, 84.6, 94. ],
      dtype=float32))
INDEX: 4
(array([8502,   24,   34,   38,   21,  208, 2556, 8634, 3121,  422]), array([  0. ,  21.6,  43.2,  64.8,  86.4, 108. , 129.6, 151.2, 172.8,
      

In [4]:
# plt.figure(figsize=(10, 5))
# plt.plot(latents[:, :])
# plt.show()

In [21]:
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.animation import FFMpegWriter
from behavenet.plotting import concat

t_beg = time.time()

frame_rate = 30

n_channels, y_pix, x_pix = ims_orig.shape[1:]
n_time, n_ae_latents = latents.shape
if len(labels_sc) > 0:
    n_labels = labels_sc.shape[1] // 2
else:
    n_labels = 0

spc = 1.1 * abs(latents.max())
plotting_latents = latents + spc * np.arange(n_ae_latents)
if len(masks) > 0:
    plotting_latents[masks != 1] = np.nan
ymin = min(-spc - 1, np.nanmin(plotting_latents))
ymax = max(spc * n_ae_latents, np.nanmax(plotting_latents))
    
n_cols = 2
n_rows = 2
offset = 2  # 0 if ims_recon_lin is None else 1
scale_ = 5
fig_width = scale_ * n_cols * n_channels / 2
fig_height = y_pix / x_pix * scale_ * n_rows / 2
fig = plt.figure(figsize=(fig_width, fig_height + offset))

gs = GridSpec(n_rows, n_cols, figure=fig)
axs = []
axs.append(fig.add_subplot(gs[0, 0]))  # 0: original frames
axs.append(fig.add_subplot(gs[0, 1]))  # 1: ae reconstructed frames
axs.append(fig.add_subplot(gs[1, :]))  # 2: latents and states
for i, ax in enumerate(fig.axes):
    ax.set_yticks([])
    if i == 2:
        ax.get_xaxis().set_tick_params(labelsize=12, direction='in')
#         ax.spines['top'].set_visible(False);
#         ax.spines['right'].set_visible(False);
#         ax.spines['left'].set_visible(False)
axs[0].set_xticks([])
axs[1].set_xticks([]); axs[1].set_axis_off()
axs[2].set_xticks([])

# lims
axs[0].set_xlim([0, x_pix])
axs[0].set_ylim([y_pix, 0])  # flip y-axes
axs[2].set_xlim([-n_time / 20, n_time + n_time / 20])

# check that the axes are correct
fontsize = 12
idx = 0
axs[idx].set_title('Video data', fontsize=fontsize); idx += 1
idx += 1
# axs[idx].set_title('Latents and states', fontsize=fontsize)
axs[idx].set_title('Labels', fontsize=fontsize)
axs[idx].set_xlabel('Time (bins)', fontsize=fontsize)

times = np.arange(n_time)

im_kwargs = {'animated': True, 'cmap': 'gray', 'vmin': 0, 'vmax': 1}
tr_kwargs = {'animated': True, 'linewidth': 2}

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

# ims is a list of lists, each row is a list of artists to draw in the
# current frame; here we are just animating one artist, the image, in
# each frame
ims = []
for i in range(n_time):

    ims_curr = []
    idx = 0

    if i % 100 == 0:
        print('processing frame %03i/%03i' % (i, ims_orig.shape[0]))

    ###################
    # behavioral videos
    ###################
    # original video
    ims_tmp = ims_orig[i, 0] if n_channels == 1 else concat(ims_orig[i])
    im = axs[idx].imshow(ims_tmp, **im_kwargs)
    ims_curr.append(im)
    # labels if they exist
    for label in range[1, 2, 4]:
        if masks[i, label] == 1:
            im = axs[idx].scatter(
                labels_sc[i, label], labels_sc[i, label+n_labels], color=colors[label])
            ims_curr.append(im)
    idx += 1

    # blank space
    idx += 1

    ################
    # latents+states
    ################
    ims_curr.append(im)
    # plot latents
    for latent in range(n_ae_latents):
        im = axs[idx].plot(
            times, plotting_latents[:, latent], color='k', **tr_kwargs)[0]
        ims_curr.append(im)
    # cover with white box
    if i + 1 < n_time:
        im = axs[idx].imshow(
            np.zeros((1, 1)), aspect='auto', 
            extent=(i+1, n_time + n_time / 25, ymin + 0.1, ymax - 0.1), 
            cmap='Greys', alpha=0.8, zorder=3)  # zorder so on top of line plots
        ims_curr.append(im)
        
    ims.append(ims_curr)

plt.tight_layout(pad=0)

ani = animation.ArtistAnimation(fig, ims, blit=True, repeat_delay=1000)
writer = FFMpegWriter(fps=frame_rate, bitrate=-1)

if save_file is not None:
    if save_file[-3:] != 'mp4':
        save_file += '.mp4'
    print('saving video to %s...' % save_file, end='')
    ani.save(save_file, writer=writer)
    print('done')
    
print('total time: %f sec' % (time.time() - t_beg))

processing frame 000/512
processing frame 100/512
processing frame 200/512
processing frame 300/512
processing frame 400/512
processing frame 500/512
saving video to /Users/johnzhou/code/behavenet/results/check.mp4...done
total time: 134.940834 sec
